In [2]:
import anywidget
import traitlets
from traitlets import List, Any
from pathlib import Path
import numpy as np
import scipy.ndimage
import ipywidgets as widgets
from IPython.display import display

grids = 2
boxs = 5

voxelarray = np.zeros((boxs * grids, boxs * grids, boxs * grids))

i = 1
for xi in range(0, 2):
    for yi in range(0, 2):
        for zi in range(0, 2):
            voxelarray[
                xi * boxs : xi * boxs + boxs,
                yi * boxs : yi * boxs + boxs,
                zi * boxs : zi * boxs + boxs,
            ] = i
            i += 1

voxelarray = np.uint8(voxelarray * 255 / i)

class HelloWidget(anywidget.AnyWidget):
    _esm = Path("widget.js")

    count = traitlets.Int(0).tag(sync=True)
    voxel_data = List(List(List(Any()))).tag(sync=True)

# Initialize the widget first
three_viewer = HelloWidget()

# Then set the voxel data
voxel_data_list = voxelarray.tolist()
three_viewer.voxel_data = voxel_data_list

# Create and display the slider
sigma_slider = widgets.FloatSlider(value=0.1, min=0, max=4, step=0.01, description='Sigma:')

def apply_gaussian_filter(sigma):
    voxelarrayX = scipy.ndimage.gaussian_filter(voxelarray, sigma=sigma)
    voxel_data_list = voxelarrayX.tolist()
    three_viewer.voxel_data = voxel_data_list
    print(f"Applied Gaussian filter with sigma: {sigma}")

def on_slider_change(change):
    apply_gaussian_filter(change['new'])

sigma_slider.observe(on_slider_change, names='value')

# Display the widgets together
w = widgets.VBox([sigma_slider, three_viewer])
display(w)
page = w